# Семинар решающие деревья



In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
titanic_df = pd.read_csv("/content/Titanic-Dataset.csv")

In [ ]:
titanic_df

In [ ]:
titanic_df['Sex'] = titanic_df['Sex'].map({'male': 0, 'female': 1})

# Базовое обучение DecisionTreeCLassifier

In [ ]:
X = titanic_df[["Age", "Sex"]]
y = titanic_df.Survived

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

DecisionTreeClassifier реализует функционал построения дерева решений

**criterion**: Функция, используемая для измерения качества разбиения. Поддерживаемые значения: "gini" для критерия Джини и "entropy" для энтропии.

**max_depth**: Максимальная глубина дерева

**min_samples_split**: Минимальное количество примеров, необходимое для выполнения разбиения узла

**min_samples_leaf**: Минимальное количество примеров, необходимое для того, чтобы узел стал листом. Помогает контролировать глубину дерева.

Обучим простой классификатор и ограничим его глубиной 2

In [ ]:
clf_titanic_depth_2 = DecisionTreeClassifier(random_state=42, max_depth=2)#, criterion="entropy")
clf_titanic_depth_2.fit(X_train, y_train)

In [ ]:
# Визуализация дерева с глубиной 2
plt.figure(figsize=(20,4))
plot_tree(clf_titanic_depth_2, filled=True, feature_names=["Age", "Sex"], class_names=["Not survived", "Survived"])
plt.title("Decision Tree with Depth 2")
plt.show()

In [ ]:
# Предскажем выживаемость пассажира
new_passenger = {
    'Age': 25.0,  # Возраст пассажира
    'Sex': 1,     # 1 - женщина, 0 - мужчина
}

new_passenger_df = pd.DataFrame([new_passenger])
clf_titanic_depth_2.predict(new_passenger_df)

In [ ]:
clf_titanic_no_depth = DecisionTreeClassifier(random_state=42)
clf_titanic_no_depth.fit(X_train, y_train)

In [ ]:
# Визуализация дерева без ограничения глубины дерева
plt.figure(figsize=(40,10))
plot_tree(clf_titanic_no_depth, filled=True, feature_names=["Age", "Sex"], class_names=["Not survived", "Survived"])
plt.title("Decision Tree without depth limit")
plt.show()

In [ ]:
def titanic_metrics_calculation(y_test, y_pred):
  print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
  print(f"Precision: {precision_score(y_test, y_pred):.4f}")
  print(f"Recall: {recall_score(y_test, y_pred):.4f}")
  print(f"F1 Score: {f1_score(y_test, y_pred):.4f}")

In [ ]:
y_pred_depth_2 = clf_titanic_depth_2.predict(X_test)
y_pred_no_depth = clf_titanic_no_depth.predict(X_test)

# Print the metrics
print("Metrics for classification with depth 2:")
titanic_metrics_calculation(y_test, y_pred_depth_2)

print("\nMetrics for classification with no depth:")
titanic_metrics_calculation(y_test, y_pred_no_depth)

Добавим еще один признак в анализ: класс каюты Pclass

In [ ]:
X = titanic_df[["Age", "Sex", "Pclass"]]
y = titanic_df.Survived

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
clf_titanic_3features = DecisionTreeClassifier(random_state=42, max_depth=10, min_samples_split=4, criterion="entropy")
clf_titanic_3features.fit(X_train, y_train)

In [ ]:
# Визуализация дерева
plt.figure(figsize=(40,10))
plot_tree(clf_titanic_3features, filled=True, feature_names=["Age", "Sex", "Pclass"], class_names=["Not survived", "Survived"])
plt.title("Decision Tree with Entropy Criterion")
plt.show()

In [ ]:
y_pred_3features = clf_titanic_3features.predict(X_test)

print("\nMetrics for Entropy Criterion:")
titanic_metrics_calculation(y_test, y_pred_3features)

In [ ]:
# Добавляем нового пассажира
new_passenger = {
    'Age': 25.0,  # Возраст пассажира
    'Sex': 0,     # 1 - женщина, 0 - мужчина
    'Pclass': 1,  # уровень каюты
}

new_passenger_df = pd.DataFrame([new_passenger])
clf_titanic_3features.predict(new_passenger_df)

### Интерпретируемость признаков

Попробуем определить важность признаков, которые мы используем.
DecisionTreeClassifier в scikit-learn позволяет считать эту важность по шагам:
1.   **Инициализация:** для каждого признака устанавливается начальное значение важности, равное нулю
2.   **Расчет уменьшения критерия:** Для каждого узла в дереве рассчитывается, насколько уменьшается критерий (например, критерий Джини или энтропия) после разбиения по данному признаку.
Уменьшение неоднородности взвешивается по количеству образцов, которые проходят через данный узел. Это гарантирует, что разбиения, затрагивающие большее количество данных, имеют большее влияние на важность признака
3. **Аккумуляция важности:** Уменьшение критерия для каждого признака суммируется по всем узлам, в которых этот признак использовался для разбиения
4. **Нормализация:** Важности всех признаков нормализуются так, чтобы их сумма равнялась единице. Это позволяет легко сравнивать вклад каждого признака в общую модель

In [ ]:
# Feature importance for Decision Tree with three features
feature_importance = clf_titanic_3features.feature_importances_
features = X.columns

plt.figure(figsize=(10, 6))
sns.barplot(x=feature_importance, y=features, orient='h')
plt.title('Feature Importance')
plt.xlabel('Importance')
plt.ylabel('Features')
plt.show()

### Джини и энтропия


### Коэффициент Джини

$$
\text{Gini}(X) = 1 - \sum_{i=1}^{K} (p_i)^2
$$

Where:
- $ K $ - количество классов.
- $ p_i $ - вероятность класса $ i $ в X

Коэффициент Джини варьируется от 0 (отсутствие неопределенности) до $ 1 - \frac{1}{K} $(максимальная неопределенность). Меньшие значения указывают на лучшие разбиения, так как они предполают большую однородность в полученных узлах.

### Энтропия

$$
\text{Entropy}(X) = -\sum_{i=1}^{K} p_i \log_2(p_i)
$$

Где:
- $ K $ - количество классов.
- $ p_i $ - вероятность класса $ i $  в X

Энтропия варьируется от 0 (отсутствие неопределенности) до $ \log_2(K) $ (максимальная неопределенность).


In [ ]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

np.random.seed(42)
data_1 = pd.DataFrame({
    'Feature1': np.random.randint(0, 200, size=200),
    'Feature2': np.random.randint(0, 200, size=200),
    'Class': np.concatenate([np.zeros(100), np.ones(100)])  # Majority class is 0
})

# Add noise to the features
data_1['Feature1'] += np.random.normal(0, 10, size=200)
data_1['Feature2'] += np.random.normal(0, 10, size=200)

X_1 = data_1[['Feature1', 'Feature2']]
y_1 = data_1['Class']

# Split the data
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1, test_size=0.3, random_state=42)

# Train decision tree with Gini
clf_gini_1 = DecisionTreeClassifier(criterion='gini', random_state=42)
clf_gini_1.fit(X_train_1, y_train_1)

# Train decision tree with Entropy
clf_entropy_1 = DecisionTreeClassifier(criterion='entropy', random_state=42)
clf_entropy_1.fit(X_train_1, y_train_1)

# Accuracy comparison
accuracy_gini_1 = accuracy_score(y_test_1, clf_gini_1.predict(X_test_1))
accuracy_entropy_1 = accuracy_score(y_test_1, clf_entropy_1.predict(X_test_1))

print("Dataset 1 - Gini Accuracy:", accuracy_gini_1)
print("Dataset 1 - Entropy Accuracy:", accuracy_entropy_1)

### Самостоятельно посчитаем джини и энтропию

In [ ]:
# Создадим искусственный набор данных для удобства
data = pd.DataFrame({
    'Age': [22, 38, 26, 35, 28],
    'Sex': [1, 0, 1, 0, 1],
    'Pclass': [1, 1, 2, 3, 1],
    'Survived': [0, 1, 1, 0, 1]
})

In [ ]:
# Подсчет коэффициента Gini
def gini_impurity(labels):
    # TO DO
    pass

In [ ]:
# Подсчет энтропии
def entropy(labels):
    # TO DO
    pass

In [ ]:
def count_criterion(left_split, right_split):
  gini_left = gini_impurity(left_split)
  gini_right = gini_impurity(right_split)
  entropy_left = entropy(left_split)
  entropy_right = entropy(right_split)
  print(f"  Split at {value}: Gini Left = {gini_left:.4f}, Gini Right = {gini_right:.4f}, Entropy Left = {entropy_left:.4f}, Entropy Right = {entropy_right:.4f}")
  print(f"gain = {gini_impurity(data['Survived']) - (len(left_split) / len(data['Survived'])) * gini_left - (len(right_split) / len(data['Survived'])) * gini_right}")

In [ ]:
# Calculate impurity for different splits
print("Gini:", gini_impurity(data["Survived"]))
print("Entropy:", entropy(data["Survived"]))
for feature in ['Age', 'Sex', 'Pclass']:
    print(f"Feature: {feature}")
    if feature == 'Sex' or feature == 'Pclass':  # Categorical feature
        for value in data[feature].unique():
            left_split = data[data[feature] == value]['Survived']
            right_split = data[data[feature] != value]['Survived']
            count_criterion(left_split, right_split)
    else:  # Numerical feature
        for value in data[feature].unique():
            left_split = data[data[feature] <= value]['Survived']
            right_split = data[data[feature] > value]['Survived']
            count_criterion(left_split, right_split)

# Задача регрессии

In [ ]:
def get_grid(data):
    x_min, x_max = data[:, 0].min() - 1, data[:, 0].max() + 1
    y_min, y_max = data[:, 1].min() - 1, data[:, 1].max() + 1
    return np.meshgrid(np.arange(x_min, x_max, 0.01),
                         np.arange(y_min, y_max, 0.01))

In [ ]:
np.random.seed(42)
data_x = np.random.normal(size=(200, 2))
data_y = np.sqrt(data_x[:, 0] ** 2 + data_x[:, 1] ** 2)

plt.figure(figsize=(8, 8))
plt.scatter(data_x[:, 0], data_x[:, 1], c=data_y, s=60, cmap='winter')
plt.title('Generated Data')
plt.show()

In [ ]:
clf = DecisionTreeRegressor(random_state=42)
clf.fit(data_x, data_y)

xx, yy = get_grid(data_x)

predicted = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

plt.figure(figsize=(8, 8))
plt.pcolormesh(xx, yy, predicted, cmap='winter')
plt.scatter(data_x[:, 0], data_x[:, 1], c=data_y, s=60, cmap='winter', edgecolor='k')

Посмотрим как будет выглядить разделение плоскости в зависимости от

*   минимального количества объектов в листе
*   максимальной глубины дерева

In [ ]:
plt.figure(figsize=(14, 14))
for i, max_depth in enumerate([2, 4, None]):
    for j, min_samples_leaf in enumerate([15, 5, 1]):
        clf = DecisionTreeRegressor(max_depth=max_depth, min_samples_leaf=min_samples_leaf)
        clf.fit(data_x, data_y)
        xx, yy = get_grid(data_x)
        predicted = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

        plt.subplot2grid((3, 3), (i, j))
        plt.pcolormesh(xx, yy, predicted, cmap='winter')
        plt.scatter(data_x[:, 0], data_x[:, 1], c=data_y, s=30, cmap='winter', edgecolor='k')
        plt.title('max_depth=' + str(max_depth) + ' | min_samples_leaf=' + str(min_samples_leaf))

Увеличение максимальной глубины и/или уменьшение минимального количества объектов выборки в листе приводит к увеличению качества на обучающей выборке и переобучению.

# Неустойчивость решающих деревьев

Решающие деревья — это алгоритмы, чувствительные к изменениям в обучающей выборке. Даже незначительные изменения в данных могут существенно повлиять на структуру итогового классификатора. Давайте рассмотрим, как структура дерева изменяется при обучении на различных 90%-х подвыборках данных.

In [ ]:
plt.figure(figsize=(20, 6))
for i in range(3):
    clf = DecisionTreeRegressor(random_state=42)

    indecies = np.random.randint(data_x.shape[0], size=int(data_x.shape[0] * 0.9))
    clf.fit(data_x[indecies], data_y[indecies])
    xx, yy = get_grid(data_x)
    predicted = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

    plt.subplot2grid((1, 3), (0, i))
    plt.pcolormesh(xx, yy, predicted, cmap='inferno')
    plt.scatter(data_x[:, 0], data_x[:, 1], c=data_y, s=30, cmap='inferno', edgecolor='k')

# Построение разбиения своими руками

In [ ]:
# Создаем небольшой датасет с 3 признаками для задачи регрессии
data = pd.DataFrame({
    'Feature1': [1, 2, 3, 4, 5, 6],
    'Feature2': [6, 5, 4, 3, 2, 1],
    'Feature3': [1, 1, 2, 2, 3, 3],
    'Target': [5.5, 5.0, 4.5, 4.0, 3.5, 3.0]
})

In [ ]:
# Функция для вычисления среднеквадратичной ошибки (MSE)
def mean_squared_error(y_true, y_pred):
    # TO DO
    pass

# Функция для вычисления уменьшения ошибки после разбиения
def reduction_in_error(left_targets, right_targets, current_mse):
    # TO DO
    pass

In [ ]:
# Функция для выбора лучшего разбиения
def best_split_regression(data, feature):
    # TO DO
    X = data[feature].values
    y = data['Target'].values

    # Сортируем значения признака и соответствующие метки
    # TO DO

    # Ищем лучшее разбиение
    # TO DO
    pass

In [ ]:
# Проход по всем признакам и выбор лучшего разбиения
features = ['Feature1', 'Feature2', 'Feature3']
best_feature = None
best_threshold = None
best_reduction = 0

for feature in features:
    threshold, reduction = best_split_regression(data, feature)
    if reduction > best_reduction:
        best_reduction = reduction
        best_feature = feature
        best_threshold = threshold

print(f"Лучший признак для разбиения: {best_feature}")
print(f"Лучший порог: {best_threshold}")
print(f"Уменьшение ошибки: {best_reduction}")